In [1]:
# sqliteのimportとDBへの接続
import sqlite3
conn = sqlite3.connect("data/wnjpn.db")
# クエリの実行結果を変数curに代入する処理、中身の確認
cur = conn.execute("select name from sqlite_master where type='table'")
for row in cur:
   print(row)
# 木構造の定義（らしい）
class node:
   def __init__(self, name, children=None):
       self.name = name  # String
       self.children = children  # List of Class node
   # 結果表示用
   def display(self, indent = 0):
       if self.children != None:
           print(' '*indent + self.name)
           for c in self.children:
               c.display(indent+1)
       else:
           print(' '*indent + self.name)
# 上位-下位の関係にある概念の抽出
hierarchy_dict = {}  # key:上位語(String), value:下位語(List of String)
n_term_set = set()  # 下位語に含まれる単語集合
cur = conn.execute("select synset1,synset2 from synlink where link='hypo'")  # 上位語-下位語の関係にあるものを抽出
for row in cur:
   b_term = row[0]
   n_term = row[1]
   if b_term not in hierarchy_dict:
       hierarchy_dict[b_term] = []
   hierarchy_dict[b_term].append(n_term)
   n_term_set.add(n_term)
print("上位語に含まれる単語の数 ： %s" % len(hierarchy_dict))
top_concepts = list(set(hierarchy_dict.keys()) - n_term_set)
print("上位語に含まれる単語の中で下位語に含まれない単語の数 ： %s" % len(top_concepts))
# synset(概念)のIDから、概念の名称に変換する辞書の作成
synset_name_dict = {}  # key:synsetのID, value:synsetの名称
cur = conn.execute("select synset,name from synset")
for row in cur:
   synset_name_dict[row[0]] = row[1]
for k,v in synset_name_dict.items():
   print("%s : %s" % (k,v))
   break
   
# 下位の語が未登録の場合、再帰的に呼ばれる関数
def recrusive_register(term, synset_name_dict, hierarchy_dict, node_tree_dict):
   for term in hierarchy_dict[term]:
       if term not in node_tree_dict:
           if term in hierarchy_dict:
               recrusive_register(term, synset_name_dict, hierarchy_dict, node_tree_dict)
               node_tree_dict[term] = node(synset_name_dict[term], [node_tree_dict[t] for t in hierarchy_dict[term]])
           else:
               node_tree_dict[term] = node(synset_name_dict[term])
# データ投入
node_tree_dict = {}
for k in top_concepts:  # 最上位の語を起点として木構造を作成
   recrusive_register(k, synset_name_dict, hierarchy_dict, node_tree_dict)
   node_tree_dict[k] = node(synset_name_dict[k], [node_tree_dict[term] for term in hierarchy_dict[k]])
# 下位-上位の関係にある概念の抽出
cur = conn.execute("select synset1,synset2 from synlink where link='hypo'")
hierarchy_dict = {}  # key:下位語(String), value:上位語(String)
for row in cur:
   b_term = row[0] #上位語
   n_term = row[1] #下位語
   b_term = synset_name_dict[b_term]
   n_term = synset_name_dict[n_term]
   
   if n_term not in hierarchy_dict:
       hierarchy_dict[b_term] = n_term
# print(hierarchy_dict)
# 以上を踏まえて辞書型のデータをpandasDFに加工してcsvファイルに
import pandas as pd
d = hierarchy_dict
df = pd.DataFrame(d.values(), index=d.keys()).T
df.T.head(100)
df.T.to_csv("data.csv",header = False)

('pos_def',)
('link_def',)
('synset_def',)
('synset_ex',)
('synset',)
('synlink',)
('ancestor',)
('sense',)
('word',)
('variant',)
('xlink',)
上位語に含まれる単語の数 ： 20008
上位語に含まれる単語の中で下位語に含まれない単語の数 ： 346
07125096-n : expletive
